## Problem formulation

In [18]:
import pandas as pd
import numpy as np
timeseries = pd.read_csv("insilico_size10_1_timeseries.tsv", sep='\t', header=0)

#the first column of timeseries is time, each experiment 21 rows
timeseries = np.array(timeseries)

In [19]:
timeseries

array([[  0.00000000e+00,   6.66511400e-01,   1.27218600e-01, ...,
          7.43098300e-01,   6.65636600e-01,   6.95063800e-01],
       [  5.00000000e+01,   3.25774800e-01,   1.21822300e-01, ...,
          7.56751500e-01,   5.55413800e-01,   7.32716700e-01],
       [  1.00000000e+02,   1.77501200e-01,   4.43587000e-02, ...,
          6.65699300e-01,   7.21103200e-01,   6.71715600e-01],
       ..., 
       [  9.00000000e+02,   5.60311700e-01,   1.38840500e-01, ...,
          6.46766400e-01,   8.13924300e-01,   7.19531900e-01],
       [  9.50000000e+02,   6.09001000e-01,   9.41102000e-02, ...,
          5.79840000e-01,   7.05499200e-01,   6.78587400e-01],
       [  1.00000000e+03,   6.46779500e-01,   1.15380400e-01, ...,
          7.45419800e-01,   6.37267700e-01,   7.40571200e-01]])

In [23]:
dif_time = np.unique(timeseries[:,0])[1] - np.unique(timeseries[:,0])[0]
M = np.unique(timeseries[:,0]).shape[0]
p = timeseries.shape[0]/M

In [25]:
timeseries = timeseries[:,1:]

In [28]:
#matrix_y - matrix of derivatives
def matrix_y(timeseries = timeseries):
    x = timeseries.copy()
    #leave zeros rows
    y = np.zeros((M*p, n))
    for i in range(p):
        for j in range(1, M):
            y[i*M+j,:] = (x[i*M+j,:] - x[i*M+j-1,:])/dif_time
    return y    

## Preprocessing

## Algorithms

In [27]:
# Q = compute_optproblem(B1, B2, E, W, l, alpha, omega, N_ITERS = 10, nu1 = 3, nu2 = 3, p = 0.7)
n = timeseries.shape[1]
def Phi_b(i, j, timeseries = timeseries):
    Fb_self = timeseries[i*M+j,:]
    Fb_connect = Fb_self/(1+Fb_self)
    Fb = np.hstack((Fb_self,Fb_connect))
    Fb = np.hstack((Fb, Fb_connect))
    I = np.eye(n)
    Phi_b = np.kron(Fb, I)
    return Phi_b
    

In [ ]:
import numpy as np

In [ ]:
# scalar shrinkage operator
# returns sgn(x) max(|x| - eps, 0)
def ScalarShrinkage(x, eps):
    return x[abs(x) < eps] = 0.0

# returns U ScalarShrinkage(sigma, eps) V^T
def S(x, eps):
    U, s, V = np.linalg.svd(x)
    return (U.dot(ScalarShrinkage(s, eps))).dot(V.T)

# need to process x
def P(x, omega):
    return x

In [ ]:
def compute_optproblem(B1, B2, E, W, l, alpha, omega, N_ITERS = 10, nu1 = 3, nu2 = 3, p = 0.7):
    def update_A(Ak, Wk, Ek, Y1k, Y2k, uk):
        return S(1 / uk, Wk - Y1k / uk)
    
    def update_W(Ak, Wk, Ek, Y1k, Y2k, uk):
        PWk = P(B1.dot(W).dot(B2.T) + Ek - D + Y2k / uk, omega)
        x = Wk - ((B1.T.dot(PWk)).dot(B2) + Wk - Ak - Y1k / uk) / nu1
        return ScalarShrinkage(x, l / (uk * nu1))
    
    def update_E(Ak, Wk, Ek, Y1k, Y2k, uk):
        PEk = P(E + (B1.dot(Wk)).dot(B2.T) - D, omega)
        x = Ek - (PEk  + Y2k / uk) / nu2
        return ScalarShrinkage(x, alpha / (uk * nu2))
    
    def update_Y1(Ak, Wk, Ek, Y1k, Y2k, uk):        
        return Y1k + uk * P((B1.dot(Wk)).dot(B2.T) + Ek - D, omega)
    
    def update_Y2(Ak, Wk, Ek, Y1k, Y2k, uk):
        return Y2k + uk * (Ak - Wk)
    
    Ak = W.copy()
    Wk = W.copy()
    Ek = E.copy()
    Y1k = 0
    Y2k = 0
    # what is u0?
    uk = 1
    
    Y1k = update_Y1(Ak, Wk, Ek, Y1k, Y2k, uk, l)
    Y2k = update_Y2(Ak, Wk, Ek, Y1k, Y2k, uk, l)
    
    for i in range(N_ITERS):
        Ak = update_A(Ak, Wk, Ek, Y1k, Y2k, uk, l)
        Wk = update_W(Ak, Wk, Ek, Y1k, Y2k, uk, l)
        Ek = update_E(Ak, Wk, Ek, Y1k, Y2k, uk, l)
        Y1k = update_Y1(Ak, Wk, Ek, Y1k, Y2k, uk, l)
        Y2k = update_Y2(Ak, Wk, Ek, Y1k, Y2k, uk, l)
        uk *= p
        
    return Wk, Ek

In [ ]:
# update omega
def supp(E):
    return omega

def comute_all(B1, B2, E, W, l, alpha, omega, N_ITERS = 10):    
    Wk = W
    Ek = E
    
    # what is omega0
    omega_k = set()
    for i in range(N_ITERS):
        Wk, Ek = compute_optproblem(B1, B2, E, W, l, alpha, omega_k)
        omega_k = omega_k.difference(supp(Ek))
        
    return Wk

## Vizualizations